In [3]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms.functional as TF
import torch.optim as optim
import BatchMaker

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import PIL
import random
import time
import os
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

PyTorch Version:  1.5.0
Torchvision Version:  0.6.0


In [8]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(9, 64,kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.BatchNorm2d(64))
    
        self.layer2 = nn.Sequential( 
            nn.Conv2d(64, 64,kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.BatchNorm2d(64))
        
        self.layer3 = nn.Sequential(
            nn.MaxPool2d(2, stride=2, padding=0))
            
        self.layer4 = nn.Sequential(
            nn.Conv2d(64,128,kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.BatchNorm2d(128))

        self.layer5 = nn.Sequential(
            nn.Conv2d(128, 128,kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.BatchNorm2d(128))
            
        self.layer6 = nn.Sequential(
            nn.MaxPool2d(2, stride=2, padding=0))
            
        self.layer7 = nn.Sequential(
            nn.Conv2d(128, 256,kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.BatchNorm2d(256))
            
        self.layer8 = nn.Sequential(
            nn.Conv2d(256, 256,kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.BatchNorm2d(256))
            
        self.layer9 = nn.Sequential(
            nn.Conv2d(256, 256,kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.BatchNorm2d(256))
            
        self.layer10 = nn.UpsamplingBilinear2d(scale_factor=2)
            
        self.layer11 = nn.Sequential(
            nn.Conv2d(256, 128,kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.BatchNorm2d(128))
            
        self.layer12 = nn.Sequential(
            nn.Conv2d(128, 128,kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.BatchNorm2d(128))
            
        self.layer13 = nn.Sequential(
            nn.Conv2d(128, 128,kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.BatchNorm2d(128))
            
        self.layer14 = nn.UpsamplingBilinear2d(scale_factor=2)
            
        self.layer15 = nn.Sequential(
            nn.Conv2d(128, 64,kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.BatchNorm2d(64))
            
        self.layer16 = nn.Sequential(
            nn.Conv2d(64, 64,kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.BatchNorm2d(64))
            
        self.layer17 = nn.Sequential(
            nn.Conv2d(64, 256,kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.BatchNorm2d(256))
            
        self.layer18 = nn.Softmax(dim=2)
            
    def forward(self,x):
        x = self.layer1(x)  
        x = self.layer2(x)  
        x = self.layer3(x)  
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.layer6(x)  
        x = self.layer7(x)
        x = self.layer8(x)
        x = self.layer9(x)  
        x = self.layer10(x)  
        x = self.layer11(x)  
        x = self.layer12(x)  
        x = self.layer13(x)  
        x = self.layer14(x)  
        x = self.layer15(x)  
        x = self.layer16(x)
        x = self.layer17(x) 
        print(x.shape())
        x = self.layer18(x)  
        return x

In [9]:
# Check if GPU is available and move the model over to GPU
if torch.cuda.is_available():
    device = torch.device("cuda: 0")
    gpu_name = torch.cuda.get_device_name()
    print(f"Running on your {gpu_name} (GPU)")
else:
    device = torch.device("cpu")
    print("Running on your CPU")

net = CNN().to(device)

Running on your GeForce GTX 1080 (GPU)


In [11]:
Generator = BatchMaker.BatchMaker

batch_size = 1
training_file = "training_model.csv"

In [20]:
loss_fn = nn.L1Loss(reduction='sum')
optimizer = optim.Adam(net.parameters(), lr=0.001)

epochs = 1
epochs_no_improve_limit = 7

train_losses = []
val_losses = []


def train():
    epochs_no_improve = 0
    min_val_loss = np.Inf
    since = time.time()
    iteration = 0
    for epoch in range(epochs):
        print('Epoch {}/{}'.format(epoch + 1, epochs))
        print('-' * 10)
        val_loss = 0
        train_loss = 0
        
        for train_inputs, train_labels in Generator(training_file, batch_size):
            optimizer.zero_grad()
            output = net(torch.from_numpy(train_inputs).transpose(0,3,1,2).to(device))
            loss = loss_fn(output.cpu(), torch.from_numpy(train_labels).cpu())
            train_loss += loss
            
#             fig = plt.figure(figsize = (32,24))
#             ax1 = fig.add_subplot(2, 2, 1)
#             ax1.set_title('Input red')
#             ax2 = fig.add_subplot(2, 2, 2)
#             ax2.set_title('Input green')
#             ax3 = fig.add_subplot(2, 2, 3)
#             ax3.set_title('Label')
#             ax4 = fig.add_subplot(2, 2, 4)
#             ax4.set_title('Output of the CNN')

#             sns.heatmap(train_inputs[0][0], vmin=0, vmax=1, cmap='gray', ax=ax1, cbar_kws={'shrink': .9})
#             sns.heatmap(train_inputs[0][1], vmin=0, vmax=1, ax=ax2, cbar_kws={'shrink': .9})
#             sns.heatmap(train_labels[0][0], vmin=0, vmax=1, ax=ax3, cbar_kws={'shrink': .9})
#             sns.heatmap(output.cpu().detach().numpy()[0], vmin=0, vmax=1, ax=ax4, cbar_kws={'shrink': .9})
#             plt.show()


#             writer.add_scalar('Loss/train', loss, iteration)

            print('Training Loss: {:.4f}'.format(loss.item()))
            loss.backward()
            optimizer.step()
            iteration += 1
        
        train_loss = train_loss / len(dataloaders['train'])
        train_losses.append(float(train_loss))
        
        del train_inputs
        del train_labels
        del output
        del loss
        torch.cuda.empty_cache()
        
#         with torch.no_grad():
#             for val_inputs, val_labels in dataloaders['val']:
#                 torch.cuda.empty_cache()
#                 output = net(val_inputs.to(device)).reshape(batch_size, output_height, output_width)
#                 loss = loss_fn(output.cpu(), val_labels.cpu().reshape(batch_size, output_height, output_width))
#                 val_loss += loss

#             val_loss = val_loss / len(dataloaders['val'])
#             val_losses.append(float(val_loss))
#             print('-' * 10)
#             print('Validation Loss: {:.4f}'.format(val_loss))

#             if val_loss < min_val_loss:
#                 torch.save({'state_dict': net.state_dict()}, 'Nets/pt-labi_CNN.pt')
#                 epochs_no_improve = 0
#                 min_val_loss = val_loss
#             else:
#                 epochs_no_improve += 1
#                 if epochs_no_improve == epochs_no_improve_limit:
#                     print('Early stopping initiated')
#                     model = torch.load('Nets/pt-labi_CNN.pt')
#                     print('Best model so far has been loaded')
    print('Least validation Loss: {:4f}'.format(min_val_loss))
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Finished training')
train()

Epoch 1/1
----------


TypeError: transpose() received an invalid combination of arguments - got (int, int, int, int), but expected one of:
 * (name dim0, name dim1)
 * (int dim0, int dim1)


In [19]:
for train_inputs, train_labels in Generator(training_file, batch_size):
    lol = torch.from_numpy(train_inputs)
    print(lol.size())
    print(train_labels)
    break

torch.Size([1, 180, 240, 9])
[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]
